In [385]:
import pandas as pd
import numpy as np
import sklearn
pd.options.mode.chained_assignment = None
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from my_plot import PrettyPlot
PrettyPlot(plt);
plt.rcParams['savefig.dpi'] = 500;

In [386]:
salaries = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Projections/salaries.csv')
rb = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Projections/rb_sampling.csv', index_col=0).reset_index(drop=True)
wr = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Projections/wr_sampling.csv', index_col=0).reset_index(drop=True)
qb = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Projections/qb_sampling.csv', index_col=0).reset_index(drop=True)
te = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Projections/te_sampling.csv', index_col=0).reset_index(drop=True)
rookie_rb = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Projections/rookie_rb_sampling.csv').reset_index(drop=True)

In [387]:
rb = pd.merge(rb, salaries, how='inner', left_on='player', right_on='player')
wr = pd.merge(wr, salaries, how='inner', left_on='player', right_on='player')
qb = pd.merge(qb, salaries, how='inner', left_on='player', right_on='player')
te = pd.merge(te, salaries, how='inner', left_on='player', right_on='player')
rookie_rb = pd.merge(rookie_rb, salaries, how='inner', left_on='player', right_on='player')

In [388]:
cols = ['player', 'salary', 'cluster', 'pred']
numbers = [str(x) for x in range(1000)]
cols.extend(numbers)

In [389]:
rb = rb[cols]
wr = wr[cols]
qb = qb[cols]
te = te[cols]
rookie_rb = rookie_rb[cols]

In [390]:
rb = rb.append(rookie_rb).reset_index(drop=True)

In [391]:
flex = rb.append(wr)
flex = flex.reset_index(drop=True)

In [392]:
qb['salary'] = qb.salary * 1.15
rb['salary'] = rb.salary * 1.15
wr['salary'] = wr.salary * 1.15
te['salary'] = te.salary * 1.15
flex['salary'] = flex.salary * 1.15

In [393]:
keepers = ['Christian McCaffrey',
           'Michael Thomas',
           'Josh Gordon',
           'Larry Fitzgerald',
           'Deshaun Watson',
           'Travis Kelce',
           'Russell Wilson',
           'Todd Gurley',
           'Devin Funchess',
           'David Johnson',
           'Alex Collins',
           'Ezekiel Elliott',
           'Kareem Hunt',
           'Jerick McKinnon',
           'Josh Gordon',
           'Allen Robinson', 
           'Julian Edelman',
           'Tyreek Hill',
           'Marvin Jones']

def drop_keepers(df, keepers):
    for player in keepers:
        idx = df[df.player == player].index
        df = df.drop(idx, axis=0).reset_index(drop=True)
    
    return df

rb = drop_keepers(rb, keepers)
wr = drop_keepers(wr, keepers)
flex = drop_keepers(flex, keepers)
te = drop_keepers(te, keepers)
qb = drop_keepers(qb, keepers)

# Functions

In [80]:
from scipy.stats import skewnorm

def grab_player(df):
    
    rindex = random.randint(0, df.shape[0]-1)
    rcol = random.randint(4, 1003)
    
    points = df.iloc[rindex, rcol] + df.iloc[rindex, 3]
    money = df.iloc[rindex, 1] + df.iloc[rindex, 1]*(skewnorm.rvs(.5, size=1)*4/100)
    name = df.iloc[rindex, 0]
    
    return(name, points, money[0])

In [117]:
def pull_specific_player(name):
    
    # specify keepers for team
    alvin_idx = flex[flex.player == 'Alvin Kamara'].index
    dalvin_idx = flex[flex.player == 'Dalvin Cook'].index

    # create temporary flex dataframe that does not contain keepers
    tmp_flex = flex.drop(alvin_idx, axis=0)
    tmp_flex = tmp_flex.drop(dalvin_idx, axis=0).reset_index(drop=True)
    
    if name == None:
        # set specific flex to tmp_flex if no specific player should be pulled
        specific_flex = tmp_flex
    else:
        # set specific flex to the player named in function input
        idx = tmp_flex[tmp_flex.player == name].index[0]
        specific_flex = pd.DataFrame([tmp_flex.iloc[idx, :], tmp_flex.iloc[idx, :]]).reset_index(drop=True)
        
        # drop specific player from tmp_flex that other players are drawn from
        tmp_flex = tmp_flex.drop(idx, axis=0).reset_index(drop=True)
        
    return tmp_flex, specific_flex

In [376]:
def sim_par(n):
        
    rcol = random.randint(4, 1002)
    alvin_pts = rb.iloc[1, rcol] + rb.iloc[1, 3]
    dalvin_pts = rb.iloc[44, rcol] + rb.iloc[44, 3]

    if (n+1) % 500000 == 0:
            print(str(datetime.datetime.now())[:-7])
            print('Completed ' + str(n+1) + ' iterations')
            
    qb_name, qb_pts, qb_money = grab_player(qb)

    rb1_name, rb1_pts, rb1_money = 'Alvin Kamara', alvin_pts, 20
    rb2_name, rb2_pts, rb2_money = 'Dalvin Cook', dalvin_pts, 31
    
    wr1_name, wr1_pts, wr1_money = grab_player(tmp_flex)
    wr2_name, wr2_pts, wr2_money = grab_player(tmp_flex)
        
    te_name, te_pts, te_money = grab_player(te)
        
    flex1_name, flex1_pts, flex1_money = grab_player(tmp_flex)            
    flex2_name, flex2_pts, flex2_money = grab_player(specific_flex)
    
    salary = qb_money + rb1_money + rb2_money + wr1_money + wr2_money + te_money+ flex1_money + flex2_money
    
    y = qb_pts + rb1_pts + rb2_pts + wr1_pts + wr2_pts + te_pts + flex1_pts + flex2_pts - np.exp(.1*(salary-293))

    if (salary > 310) | (salary < 250) | (y < 115):
        return
            
    sort_money = pd.Series([qb_money, rb1_money, rb2_money, wr1_money, wr2_money, te_money, flex1_money, flex2_money],
                                index = [qb_name, rb1_name, rb2_name, wr1_name, wr2_name, te_name, flex1_name, flex2_name]).sort_values(ascending=False)
    
    # 1. extend list w/ player names, 2. append total points, 3. append total salary, 4. extend salary values
    results = list(sort_money.index)
    results.append(y)
    results.append(salary)
    results.extend(sort_money.values)
        
    return results

In [84]:
from multiprocessing import Pool
import time
import random

def run_sim_par(iterations):
    
    p = Pool(processes=4)

    t1 = time.time()
    
    result = p.map(sim_par, range(iterations))
    p.close()
    p.join()
    
    df = pd.DataFrame([i for i in result if i is not None])
    df.columns = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'ppg', 'total_salary',
                  'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8']
    
    print('Time Elapsed: ' + str(round(time.time()-t1,2)))

    return df

In [403]:
def show_results(results, cols, added_cost=0):
    
    # plot histogram and number of results that made it
    print('')
    print('o Number of Results Above 115 PPG:', results.shape[0])
    print('')
    print('')                           
    
    for col in cols:
        
        counts = pd.DataFrame(results.loc[:,'p'+col].value_counts()[:15])
        money = pd.DataFrame(results.groupby('p'+col)['m'+col].agg(np.mean).astype('int'))

        combined = pd.merge(counts, money, how='inner', left_index=True, right_index=True)
        combined.columns = ['Counts', 'Avg_Cost']  
        combined['Remaining $'] = 300 - (combined.Avg_Cost + added_cost + 30 + 21).astype('int')
        
        print('o Most Common ' + '#' + col + ' Players')
        print('------------------------')
        display(combined)
        print('')
        print('')

# Standard Simulation

In [401]:
tmp_flex, specific_flex = pull_specific_player(None)
df = run_sim_par(3000000)

2018-08-26 12:20:12
Completed 500000 iterations
2018-08-26 12:23:48
Completed 1000000 iterations
2018-08-26 12:27:55
Completed 1500000 iterations
2018-08-26 12:31:41
Completed 2000000 iterations
2018-08-26 12:38:30
Completed 2500000 iterations
2018-08-26 12:45:35
Completed 3000000 iterations
Time Elapsed: 1752.65


In [402]:
show_results(df, ['1', '2', '3', '4'], added_cost=0)


o Number of Results Above 115 PPG: 200


o Most Common #1 Players
------------------------
                 Counts  Avg_Cost  Remaining $
Le'Veon Bell         42       115          134
Antonio Brown        36       108          141
DeAndre Hopkins      26        99          150
Devonta Freeman      15        79          170
Keenan Allen         15        90          159
Julio Jones          13       104          145
Odell Beckham        13        96          153
Saquon Barkley       10       106          143


o Most Common #2 Players
------------------------
                   Counts  Avg_Cost  Remaining $
Rob Gronkowski         30        76          173
Joe Mixon              30        66          183
Mike Evans             13        74          175
Melvin Gordon          11        87          162
Leonard Fournette      10        90          159
Odell Beckham          10        97          152
Devonta Freeman        10        80          169
Davante Adams           9        77      

## Le'Veon First Pick

In [377]:
tmp_flex, specific_flex = pull_specific_player("Le'Veon Bell")
lev = run_sim_par(3000000)

2018-08-26 10:50:09
Completed 500000 iterations
2018-08-26 10:53:33
Completed 1000000 iterations
2018-08-26 10:56:57
Completed 1500000 iterations
2018-08-26 11:00:21
Completed 2000000 iterations
2018-08-26 11:03:44
Completed 2500000 iterations
2018-08-26 11:07:11
Completed 3000000 iterations
Time Elapsed: 1225.86


In [378]:
show_results(lev, ['2', '3', '4'], added_cost=115)


o Number of Results Above 115 PPG: 2255


o Most Common #2 Players
------------------------
                 Counts  Avg_Cost  Remaining $
Rob Gronkowski      294        76           58
Joe Mixon           189        66           68
Keenan Allen        131        91           43
Devonta Freeman     125        80           54
Zach Ertz           119        55           79
Melvin Gordon        99        88           46
Odell Beckham        98        96           38
DeAndre Hopkins      90        99           35


o Most Common #3 Players
------------------------
               Counts  Avg_Cost  Remaining $
Dalvin Cook      1850        31          103
Kenyan Drake      120        39           95
Brandin Cooks      60        36           98
Royce Freeman      55        34          100
Jarvis Landry      28        42           92
Ronald Jones       21        30          104
Amari Cooper       18        44           90
Jay Ajayi          17        31          103


o Most Common #4 Players


## Le'Veon First Pick + Require WR

In [241]:
tmp_flex, specific_flex = pull_specific_player("Le'Veon Bell")
lev_wr = run_sim_par(3000000)

2018-08-26 08:19:16
Completed 500000 iterations
2018-08-26 08:22:40
Completed 1000000 iterations
2018-08-26 08:26:06
Completed 1500000 iterations
2018-08-26 08:29:32
Completed 2000000 iterations
2018-08-26 08:33:29
Completed 2500000 iterations
2018-08-26 08:37:32
Completed 3000000 iterations
Time Elapsed: 1301.4


In [407]:
show_results(combined_lev, ['2', '3', '4', '5'], added_cost = 115)


o Number of Results Above 115 PPG: 1788


o Most Common #2 Players
------------------------


,Counts,Avg_Cost,Remaining $
Rob Gronkowski,160,76,58
Keenan Allen,125,91,43
Odell Beckham,111,97,37
DeAndre Hopkins,108,98,36
Joe Mixon,106,66,68
Mike Evans,104,74,60
Zach Ertz,77,55,79
Davante Adams,73,77,57
T.Y. Hilton,70,62,72
Demaryius Thomas,66,52,82




o Most Common #3 Players
------------------------


,Counts,Avg_Cost,Remaining $
Dalvin Cook,1458,31,103
Kenyan Drake,72,39,95
Jarvis Landry,57,42,92
Brandin Cooks,53,36,98
Amari Cooper,30,44,90
Royce Freeman,25,34,100
Alshon Jeffery,18,38,96
Ronald Jones,15,31,103
Golden Tate,11,47,87
Demaryius Thomas,10,51,83




o Most Common #4 Players
------------------------


,Counts,Avg_Cost,Remaining $
Alvin Kamara,866,20,114
Dalvin Cook,307,31,103
Aaron Rodgers,145,21,113
JuJu Smith-Schuster,118,25,109
Mark Ingram,47,20,114
Tom Brady,46,20,114
Robert Woods,45,24,110
Michael Crabtree,42,23,111
Ronald Jones,41,30,104
Chris Hogan,40,23,111




o Most Common #5 Players
------------------------


,Counts,Avg_Cost,Remaining $
Alvin Kamara,725,20,114
Aaron Rodgers,116,20,114
Tom Brady,103,19,115
Evan Engram,84,15,119
Mark Ingram,82,19,115
Carson Wentz,52,9,125
Rashaad Penny,47,7,127
Cam Newton,43,9,125
Jimmy Graham,43,10,124
Corey Davis,42,13,121


## AB First Pick

In [ ]:
tmp_flex, specific_flex = pull_specific_player('Antonio Brown')
combined_ab = run_sim_par(3000000)

In [370]:
show_results(combined_ab, ['1', '2', '3', '4'], added_cost = 108)


o Number of Results Above 115 PPG: 1707


o Most Common #1 Players
------------------------
                 Counts  Avg_Cost  Remaining $
Antonio Brown      1620       108           33
Le'Veon Bell         57       114           27
Saquon Barkley       14       107           34
Julio Jones          10       106           35
DeAndre Hopkins       5       102           39
Keenan Allen          1       102           39


o Most Common #2 Players
------------------------
                   Counts  Avg_Cost  Remaining $
Rob Gronkowski        200        76           65
Joe Mixon             140        66           75
Keenan Allen          104        91           50
Devonta Freeman       100        80           61
Leonard Fournette      96        90           51
Melvin Gordon          94        88           53
DeAndre Hopkins        92        98           43
Antonio Brown          87       106           35


o Most Common #3 Players
------------------------
                Counts  Avg_Cost 

## DeAndre First Pick

In [ ]:
tmp_flex, specific_flex = pull_specific_player('DeAndre Hopkins')
combined_deandre = run_sim_par(3000000)

In [374]:
show_results(combined_deandre, ['1', '2', '3', '4'], added_cost = 100)


o Number of Results Above 115 PPG: 1516


o Most Common #1 Players
------------------------
                   Counts  Avg_Cost  Remaining $
DeAndre Hopkins      1203       100           49
Le'Veon Bell           90       114           35
Antonio Brown          77       106           43
Saquon Barkley         50       106           43
Julio Jones            46       103           46
Odell Beckham          35       100           49
Keenan Allen            8        96           53
Leonard Fournette       5        95           54


o Most Common #2 Players
------------------------
                   Counts  Avg_Cost  Remaining $
DeAndre Hopkins       313        98           51
Rob Gronkowski        217        77           72
Joe Mixon             128        66           83
Devonta Freeman        94        79           70
Melvin Gordon          90        88           61
Keenan Allen           76        91           58
Mike Evans             75        75           74
Leonard Fournette     

## Odell First Pick

In [ ]:
tmp_flex, specific_flex = pull_specific_player('Odell Beckham')
combined_obj = run_sim_par(3000000)

In [375]:
show_results(combined_obj, ['1', '2', '3', '4'], added_cost=97)


o Number of Results Above 115 PPG: 1400


o Most Common #1 Players
------------------------
                   Counts  Avg_Cost  Remaining $
Odell Beckham        1038        98           54
Antonio Brown          86       107           45
Le'Veon Bell           84       114           38
DeAndre Hopkins        69       101           51
Saquon Barkley         61       105           47
Julio Jones            43       102           50
Keenan Allen           11        95           57
Leonard Fournette       6        97           55


o Most Common #2 Players
------------------------
                   Counts  Avg_Cost  Remaining $
Odell Beckham         362        96           56
Rob Gronkowski        187        76           76
Joe Mixon             109        66           86
Keenan Allen           94        91           61
Devonta Freeman        78        80           72
Melvin Gordon          75        87           65
Leonard Fournette      61        90           62
Zach Ertz             

# Late Round Dudes

# Create Visualizations

In [ ]:
names = ["L.Bell", 'M.Evans', 'D.Adams', 'TY.Hilton', 'J.Mixon', 'AJ.Green', 'M.Gordon']

In [ ]:
names_evans = ['', '', '', '', '', '', '',
               'B.Cooks', 'A.Cooper', 'D.Baldwin', 'K.Drake', 'J.Landry']

In [ ]:
names_adams = ['', '', '', '', '', '', '', 
               '', '', '', '', '', 
               'B.Cooks', 'J.Landry', 'A.Cooper', 'K.Drake', 'D.Henry']

In [ ]:
names_hilton = ['', '', '', '', '', '', '',
                '', '', '', '', '', 
                '', '', '', '', '', 
                'B.Cooks', 'J.Landry', 'D.Baldwin', 'G.Tate', 'A.Cooper']

In [ ]:
names_mixon  = ['', '', '', '', '', '', '',
                '', '', '', '', '', 
                '', '', '', '', '',
                '', '', '', '', '',
                'B.Cooks', 'J.Landry', 'D.Baldwin', 'A.Cooper', 'D.Thomas']

In [ ]:
names_green  = ['', '', '', '', '', '', '',
                '', '', '', '', '', 
                '', '', '', '', '',
                '', '', '', '', '',
                '', '', '', '', '',
                'JuJu', 'A.Rodgers', 'M.Crabtree', 'E.Sanders', 'R.Woods']

In [ ]:
names_melvin  = ['', '', '', '', '', '', '',
                 '', '', '', '', '', 
                 '', '', '', '', '',
                 '', '', '', '', '',
                 '', '', '', '', '',
                 '', '', '', '', '',
                'JuJu', 'A.Rodgers', 'C.Hogan', 'M.Crabtree', 'R.Woods']

In [ ]:
names_evans_cooks  = ['', '', '', '', '', '', '',
                 '', '', '', '', '', 
                 '', '', '', '', '',
                 '', '', '', '', '',
                 '', '', '', '', '',
                 '', '', '', '', '',
                 '', '', '', '', '',
                'C.Newton', 'R.Anderson', 'W.Fuller', 'D.Brees']

In [ ]:
combined_3[(combined_3.p2 == 'Mike Evans') & (combined_3.p3 == 'Kenyan Drake')].p6.value_counts()[:5]

In [ ]:
all_greens = [(56,144,161), (63,161,180), (76,173,192), (95,182,199), (114,191,206), (133,199,212), (152,208,219)]
greens = ['', '', '', '', '', '', '',
          (50,138,152), (76,173,192), (95,182,199), (133,199,212), (160,220,227)]

all_yellows=[(255,188,13), (255,195,38), (255,202,64), (255,209,89), (255,216,115), (255,223,140), (255,230,166)]
yellows=[(0,0,0), (255,188,13), (255,195,38), (255,202,64), (255,209,89), (255,216,115), (255,223,140)]

all_reds = [(189,81,4), (213,92,5), (238,103,5), (250,116,19), (251,131,44), (251,145,69), (252,160,94)]
reds=[(50,50,50), (180,71,3), (213,92,5), (250,116,19), (251,145,69), (252,160,94), (254,180,112)]

In [ ]:
from pygraphviz import *

A=AGraph()

# set some default node attributes
A.node_attr['style']='filled'
A.node_attr['shape']='circle'
A.node_attr['fixedsize']='true'
A.node_attr['fontcolor']='#FFFFFF'

for i in range(7):
    A.add_edge(0,i)
    n=A.get_node(i)
    n.attr['fillcolor']='#%02x%02x%02x' % reds[i]
    n.attr['width']= 1
    n.attr['label'] = names[i]
    
# mike evans
for i in range(7,12):
    A.add_edge(1,i)
    n=A.get_node(i)
    n.attr['fillcolor']='#%02x%02x%02x' % greens[i]
    n.attr['width']= 1
    n.attr['label'] = names_evans[i]
    
# davante adams
for i in range(12,17):
    A.add_edge(2,i)
    n=A.get_node(i)
    n.attr['fillcolor']='#%02x%02x%02x' % greens[i-5]
    n.attr['width']= 1
    n.attr['label'] = names_adams[i]
    
# ty hilton
for i in range(17,22):
    A.add_edge(3,i)
    n=A.get_node(i)
    n.attr['fillcolor']='#%02x%02x%02x' % greens[i-10]
    n.attr['width']= 1
    n.attr['label'] = names_hilton[i]
    
# joe mixon
for i in range(22,27):
    A.add_edge(4,i)
    n=A.get_node(i)
    n.attr['fillcolor']='#%02x%02x%02x' % greens[i-15]
    n.attr['width']= 1
    n.attr['label'] = names_mixon[i]
    
# aj green
for i in range(27,32):
    A.add_edge(5,i)
    n=A.get_node(i)
    n.attr['fillcolor']='#%02x%02x%02x' % greens[i-20]
    n.attr['width']= 1
    n.attr['label'] = names_green[i]
    
# melvin gordon
for i in range(32,37):
    A.add_edge(6,i)
    n=A.get_node(i)
    n.attr['fillcolor']='#%02x%02x%02x' % greens[i-25]
    n.attr['width']= 1
    n.attr['label'] = names_melvin[i]
    

#print(A.string()) # print to screen
A.draw('star.pdf',prog="circo") # draw to png using circo

In [ ]:
# evans - cooks
for i in range(37,41):
    A.add_edge(7,i)
    n=A.get_node(i)
    #n.attr['fillcolor']='#%02x%02x%02x' % greens[i-25]
    n.attr['width']= 1
    n.attr['label'] = names_evans_cooks[i]
    
# evans - cooks
for i in range(41,45):
    A.add_edge(8,i)
    n=A.get_node(i)
    #n.attr['fillcolor']='#%02x%02x%02x' % greens[i-25]
    n.attr['width']= 1
    n.attr['label'] = names_evans_cooks[i-4]

In [ ]:
combined_3.p4.value_counts()[2:10]